In [1]:
from transformers import CLIPModel, CLIPTokenizerFast, CLIPImageProcessor, CLIPProcessor
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss
from dataset import TextImagePairSet
from torchvision import transforms
from torchvision.io import read_image
from torch.optim import AdamW
from tqdm import tqdm
import pandas as pd
import numpy as np
import torch

/home/antony/Documents/Visual Studio Code/clip/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/antony/Documents/Visual Studio Code/clip/env/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/antony/Documents/Visual Studio Code/clip/env/lib/python3.10/site-packages/torch/cuda/__init__.py:749: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() if nvml_count < 0 else nvml_count


In [2]:
clip:CLIPModel = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
tokenizer:CLIPTokenizerFast = CLIPTokenizerFast.from_pretrained("openai/clip-vit-base-patch32")
vision_processor:CLIPImageProcessor = CLIPImageProcessor.from_pretrained("openai/clip-vit-base-patch32")
processor:CLIPProcessor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [3]:
for x in clip.parameters():
    x.requires_grad = False

In [ ]:
clip

In [ ]:
print(sum(x.numel() for x in clip.visual_projection.parameters()))
print(sum(x.numel() for x in clip.text_projection.parameters()))

In [4]:
trainset = TextImagePairSet("./data/train/data.csv", "./data/train")
devset = TextImagePairSet("./data/dev/data.csv", "./data/dev")
testset = TextImagePairSet("./data/test/data.csv", "./data/test")

trainLoader = DataLoader(trainset, batch_size=1)
devLoader = DataLoader(devset, batch_size=1)
testLoader = DataLoader(testset, batch_size=1)

In [5]:
loss_fn = torch.nn.CrossEntropyLoss()
fc = torch.nn.Linear(in_features=768 + 512, out_features=1, bias=True)
optimizer = AdamW(fc.parameters(), 1e-5)

In [ ]:
x = None
l = []

In [ ]:

def train(trainLoader, clip, fc, optimizer, loss_fn):
    epoch_loss = 0
    fc.train()
    for X, y, label in trainLoader:                       
        X, tok_label = vision_processor(X, return_tensors="pt"), tokenizer(label, padding=True, return_tensors="pt")
        l.append(X['pixel_values'])
        optimizer.zero_grad()
        clip_outputs = clip(**X, **tok_label)
        
        y_hat = torch.nn.Softmax(dim=0)(clip_outputs['logits_per_image'])
       
        loss = loss_fn(y_hat, torch.Tensor([[y]]))
        
        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
    return epoch_loss        

In [ ]:
def validate(devLoader, clip, fc, loss_fn):
    epoch_loss = 0
    fc.eval()
    with torch.no_grad():
        for X, y, label in devLoader:            
            X, tok_label = vision_processor(X, return_tensors="pt"), tokenizer(label, padding=True, return_tensors="pt")
            clip_outputs = clip(**X, **tok_label)
            y_hat = torch.nn.Softmax(dim=0)(clip_outputs['logits_per_image'])
            loss = loss_fn(y_hat, y)
            loss.backward()
            
            epoch_loss += loss.item()
    return epoch_loss        

In [ ]:
def test(testLoader, clip, fc, loss_fn):
    epoch_loss = 0
    fc.eval()
    with torch.no_grad():
        for X, y, label in testLoader:
            X, tok_label = vision_processor(X, return_tensors="pt"), tokenizer(label, padding=True, return_tensors="pt")
            clip_outputs = clip(**X, **tok_label)
            y_hat = torch.nn.Softmax(dim=0)(clip_outputs['logits_per_image'])
            
            loss = loss_fn(y_hat, y)
            loss.backward()            

            epoch_loss += loss.item()
    return epoch_loss       


In [ ]:
epochs = 10

train_loss = [0]*10
dev_loss = [0]*10
cos_sim_fn = torch.nn.CosineSimilarity(dim=0)

for i in tqdm(range(epochs)):
    print("antes")
    train_loss[i] = train(trainLoader, clip, fc, optimizer, loss_fn)
    print("depois")
    dev_loss[i] = validate(devLoader, clip, fc, loss_fn)    

In [6]:
for i, _ in enumerate(trainLoader):
    print(i)

0
1
2
3


In [ ]:
print(le_image.shape)
es_una_label = tokenizer(["texto maluco que eu nao sei"], padding=True, return_tensors="pt")

In [ ]:
es_una_label.keys()

In [ ]:
clip_outputs = clip(**{"pixel_values" : le_image}, **es_una_label)

In [ ]:
clip_outputs.keys()
#y_hat = torch.nn.Softmax(dim=0)(clip_outputs['logits_per_image'])

In [ ]:
torch.max(clip_outputs['vision_model_output']['last_hidden_state'])

In [ ]:
print(clip_outputs['logits_per_image'])

In [ ]:
clip_outputs['logits_per_image'].shape

In [8]:
x = read_image("./data/train/1.png")

In [9]:
#inputs = processor(text="mulher em um fundo verde fazendo sinais de libras", images=x, return_tensors="pt", padding=True)
x_pross = vision_processor(x, return_tensors="pt")
tok_y = tokenizer(["mulher em um fundo verde fazendo sinais de libras"], padding=True, return_tensors="pt")
outputs = clip(**x_pross, **tok_y)

In [10]:
outputs.keys()

odict_keys(['logits_per_image', 'logits_per_text', 'text_embeds', 'image_embeds', 'text_model_output', 'vision_model_output'])

In [19]:
outputs['logits_per_image'].shape[1]

torch.Size([1, 1])

In [21]:
bl = torch.nn.Linear(in_features=1, out_features=1)
z = bl(outputs['logits_per_image'])

In [33]:
outputs['vision_model_output'].keys()

odict_keys(['last_hidden_state', 'pooler_output'])

In [11]:
softmax = torch.nn.Softmax(dim=0)
y_hat = softmax(outputs['logits_per_image'])

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()

fc = torch.nn.Linear(in_features=768 + 512, out_features=1, bias=True)
optimizer = AdamW(fc.parameters(), 1e-5)

In [ ]:
y_text = softmax(outputs['logits_per_text'])

In [ ]:
t = torch.Tensor([1, 2, 3 ,4])
t2 = torch.Tensor([5, 6, 7, 8])

In [ ]:
t_norm = t/t.norm()
print(t_norm)

t2_norm = t2/t2.norm()
t2_norm

In [ ]:
t1_t2_norm = t_norm * t2_norm
norm_t1_t2 = t_norm.norm() * t2_norm.norm()
print(t1_t2_norm)
print(norm_t1_t2)

In [ ]:
cos = t1_t2_norm / norm_t1_t2
print(cos)
sum(cos)

In [ ]:
similarity = torch.nn.CosineSimilarity(dim=0)
torch_cos = similarity(t, t2)
torch_cos

In [ ]:
(t * t2)/max(t/t.norm() * t2/t2.norm())